<a href="https://colab.research.google.com/github/tmckim/materials-fa23-colab/blob/main/lectures/lec09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before you start - Save this notebook!

When you open a new Colab notebook from the WebCampus (like you hopefully did for this one), you cannot save changes. So it's  best to store the Colab notebook in your personal drive `"File > Save a copy in drive..."` **before** you do anything else.

The file will open in a new tab in your web browser, and it is automatically named something like: "**Copy of lec09.ipynb**". You can rename this to just the title of the assignment "**lec09.ipynb**". Make sure you do keep an informative name (like the name of the assignment) so that you know which files to submit back to WebCampus for grading! More instructions on this are at the end of the notebook.


**Where does the notebook get saved in Google Drive?**

By default, the notebook will be copied to a folder called “Colab Notebooks” at the root (home directory) of your Google Drive. If you use this for other courses or personal code notebooks, I recommend creating a folder for this course and then moving the assignments AFTER you have completed them.

In [ ]:
# Setup and add files needed to gdrive
# If you restart colab, start by rerunning this cell first!
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#!mkdir -p '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/'
%cd /content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/
!rm -r materials-fa23-colab

!git clone https://github.com/tmckim/materials-fa23-colab '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/'

%cd /content/gdrive/MyDrive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/lectures/

In [ ]:
# Import packages and other things needed
# Don't change this cell; Just run this cell
# If you restart colab, make sure to run this cell again after the first one above^

from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Lecture 9 ##

## Alameda County Jury Panels ##

In [ ]:
jury = Table().with_columns(
    'Ethnicity', make_array('Asian', 'Black', 'Latino', 'White', 'Other'),
    'Eligible', make_array(0.15, 0.18, 0.12, 0.54, 0.01), # proportion from population
    'Panels', make_array(0.26, 0.08, 0.08, 0.54, 0.04)    # actual proportions from the panel- empirical data
)

jury

In [ ]:
# Plot the data- what is a good chart to compare these?
jury.barh('Ethnicity')

In [ ]:
# Under the model, this is the true distribution of people (population)
# from which the jurors are randomly sampled
# eligible column of the table
model = make_array(0.15, 0.18, 0.12, 0.54, 0.01)

In [ ]:
# Let's simulate a random draw of 1423 jurors (sample size) from this distribution
simulated = sample_proportions(1423, model)
simulated

In [ ]:
# The actual observed distribution (Panels) looks quite different
# from the simulation -- try running this several times to confirm!
jury_with_simulated = jury.with_column('Simulated', simulated)
jury_with_simulated

In [ ]:
# Visualize this new table
jury_with_simulated.barh('Ethnicity')

In [ ]:
# In the last lecture, the difference between observed white/purple
# and their expected values (25%/75%) was our statistic.

# In this case, we need to understand how each of the 5 categories
# differ from their expected values according to the model.

# How do we summarize this data into a single number?

## Distance Between Distributions

In [ ]:
# Calculate the difference
diffs = jury.column('Panels') - jury.column('Eligible')
diffs

## Total Variation Distance

In [ ]:
# Define a function so that we can repeat this process many times easily
def tvd(dist1, dist2):
    return sum(abs(dist1 - dist2))/2

In [ ]:
# The TVD of our observed data (Panels) from their expected values
# assuming the model is true (Eligible)
obsvd_tvd = tvd(jury.column('Panels'), jury.column('Eligible'))
obsvd_tvd

In [ ]:
# Define a function to help with simulation

def simulated_tvd():
    simulated_sample = sample_proportions(1423,model)
    return tvd(simulated_sample)

In [ ]:
# Another way to write the function above
def simulated_tvd():
    return tvd(sample_proportions(1423, model), model)

In [ ]:
# Simulate
# make an empty array to append to
# use a for loop and simulate using our function from above^
# take the output values from the function and add them to our array
tvds = make_array()

num_simulations = 10000
for i in np.arange(num_simulations):
    new_tvd = simulated_tvd()
    tvds = np.append(tvds, new_tvd)

In [ ]:
# Now we plot the simulated data
title = 'Simulated TVDs (if model is true)'
bins = np.arange(0, .05, .005)

Table().with_column(title, tvds).hist(bins = bins)
print('Observed TVD: ' + str(obsvd_tvd))

## The GSI's Defense ##

In [ ]:
# Read in our table of data
# Each row is a student
scores = Table.read_table('scores_by_section.csv')
scores

In [ ]:
# How many people are in each section (all sections)?
# Get the sample size


In [ ]:
# Get the average score per section


In [ ]:
# Average for section 3


In [ ]:
# Single simulation
random_sample = scores.sample(27, with_replacement=False)
random_sample

In [ ]:
# Average score for our simulated sample
np.average(random_sample.column('Midterm'))

In [ ]:
# Simulate one value of the test statistic
# under the hypothesis that the section is like a random sample from the class

def random_sample_midterm_avg():
    # Same code as above, but put into the function
    # individual sample
    # return the data

In [ ]:
def random_sample_midterm_avg():
    random_sample = scores.sample(27, with_replacement = False)
    return np.average(random_sample.column('Midterm'))

In [ ]:
# Simulate 50,000 values of the test statistic

sample_averages = make_array()

for i in np.arange(50000):
    sample_averages = np.append(sample_averages, random_sample_midterm_avg())

## Our Decision Using a Visualization

In [ ]:
# Compare the simulated distribution of the statistic
# and the actual observed statistic
averages_tbl = Table().with_column('Random Sample Average', sample_averages)
averages_tbl.hist(bins = 20)

plots.scatter(observed_average, -0.01, color='red', s=120); # plot observed average- you don't need to know this, this is extra

What would you conclude?<br>
Is the data consistent with the model or the alternative?

### Approach 1

How do we determine what percentage of our array (sample_averages) is less than the observed value?

In [ ]:
# (1) Calculate the p-value: simulation area beyond observed value
np.count_nonzero(sample_averages <= observed_average) / 50000
# (2) See if this is less than 5%- not quite small enough

### Approach 2

What value of the test statistic correspond to %5?

In [ ]:
# (1) Find simulated value corresponding to 5% of 50,000 = 2500


In [ ]:
# (2) See if this value is greater than observed value
observed_average

### Visual Representation

In [ ]:
averages_tbl.hist(bins = 20)
plots.plot([five_percent_point, five_percent_point], [0, 0.35], color='gold', lw=2) # cutoff value of 5% (extra, don't need to know)
plots.title('Area to the left of the gold line: 5%'); # title
plots.scatter(observed_average, -0.01, color='red', s=120);  # add the observed data point